# 0. 라이브러리

In [189]:
import pandas as pd
import numpy as np

In [190]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# 1. ECOS API

In [191]:
from PublicDataReader import Ecos
api_key = '37EJD8646GTPZKOYBEOD'
api = Ecos(api_key)

In [192]:
datals = api.get_statistic_table_list()
datals.head(2)

,상위통계표코드,통계표코드,통계명,시점,검색가능여부,출처
0,*,0000000001,1. 통화/금융,None,N,None
1,0000000001,0000000002,1.1. 통화/유동성,None,N,None


In [193]:
datals[datals.통계명.str.contains('GDP')]

,상위통계표코드,통계표코드,통계명,시점,검색가능여부,출처
144,0000000067,0000000071,"2.1.2. 생산 및 지출 GDP, GDP 디플레이터",None,N,None
145,0000000071,0000000072,2.1.2.1. 경제활동별 GDP 및 GNI,None,N,None
146,0000000072,200Y003,"2.1.2.1.1. 경제활동별 GDP 및 GNI(계절조정, 명목, 분기)",Q,Y,한국은행
147,0000000072,200Y004,"2.1.2.1.2. 경제활동별 GDP 및 GNI(계절조정, 실질, 분기)",Q,Y,한국은행
148,0000000072,200Y005,"2.1.2.1.3. 경제활동별 GDP 및 GNI(원계열, 명목, 분기 및 연간)",Q,Y,한국은행
149,0000000072,200Y006,"2.1.2.1.4. 경제활동별 GDP 및 GNI(원계열, 실질, 분기 및 연간)",Q,Y,한국은행
155,0000000071,0000000074,2.1.2.3. GDP 디플레이터,None,N,None
174,0000000067,0000000084,2.1.5. GDP에 대한 성장기여도,None,N,None
823,0000000444,902Y016,9.1.4.2. 국제 주요국 국내총생산(GDP),A,Y,OECD
826,0000000444,902Y018,9.1.4.5. 국제 주요국 1인당 GDP,A,Y,OECD


# 2. KOSIS 데이터

In [194]:
# GDP 국내총생산(실질성장률)

gdp = pd.read_csv('./data/kosis/GDP실질성장률.csv')
gdp = gdp.loc[gdp.국가 == '대한민국', gdp.columns[1:]]
gdp

,2015,2016,2017,2018,2019,2020,2021,2022
1,2.809,2.947,3.16,2.907,2.244,-0.709,4.305,2.613


In [195]:
# 원/달러 환율 (USD)

usd = pd.read_csv('./data/kosis/미달러화환율.csv')
usd = usd.iloc[1:, 1:]
usd

,2015,2016,2017,2018,2019,2020,2021,2022
1,1131,1160.8,1131,1100.2,1165.4,1180.3,1144,1291.4


In [196]:
# 소비자물가지수 등락

dcpi = pd.read_csv('./data/kosis/소비자물가지수등락률.csv')
dcpi = dcpi.iloc[:, 1:]
dcpi

,2015,2016,2017,2018,2019,2020,2021,2022
0,0.7,1,1.9,1.5,0.4,0.5,2.5,5.1


# 3. ECOS 데이터

In [197]:
# 생산자물가지수 등락

dppi = pd.read_csv('./data/kosis/생산자물가지수등락률.csv')[['변환', '전기대비증감률']].set_index('변환')
dppi

,전기대비증감률
변환,
2015,-4.0
2016,-1.8
2017,3.5
2018,1.9
2019,0.0
2020,-0.5
2021,6.4
2022,8.4


In [198]:
ecos_y = pd.read_csv('./data/kosis/ecos_y.csv')
ecos_y.columns = ['연도', '무담보콜금리', 'CD91일', '국고채3년', '코스피종가', '코스닥종가', '경상수지', '전산업생산지수', '실업률']
ecos_y.set_index(['연도'], inplace=True)
ecos_y

,무담보콜금리,CD91일,국고채3년,코스피종가,코스닥종가,경상수지,전산업생산지수,실업률
연도,,,,,,,,
2015,1.65,1.76,1.794,"1,961.31",682.35,"105,118.60",93.2,3.6
2016,1.34,1.49,1.442,"2,026.46",631.44,"97,923.70",96.0,3.7
2017,1.26,1.44,1.801,"2,467.49",798.42,"75,230.90",98.5,3.7
2018,1.52,1.68,2.099,"2,041.04",675.65,"77,466.50",100.1,3.8
2019,1.59,1.69,1.529,"2,197.67",669.83,"59,676.10",101.1,3.8
2020,0.70,0.92,0.988,"2,873.47",968.42,"75,902.20",100.0,4.0
2021,0.61,0.85,1.392,"2,977.65","1,033.98","85,228.20",105.3,3.7
2022,2.03,2.49,3.199,"2,236.40",679.29,"29,830.90",110.1,2.9


In [199]:
ecos_m = pd.read_csv('./data/kosis/ecos_m.csv')
ecos_m.rename(columns={'변환':'연월'}, inplace=True)
ecos_m.연월 = pd.to_datetime(ecos_m.연월)
ecos_m['연도'] = ecos_m.연월.dt.year
ecos_m.head()

,연월,경제심리지수,선행종합지수,동행종합지수,후행종합지수,뉴스심리지수,주택매매가격지수,연도
0,2015-01-01,97.8,82.2,87.5,85.1,104.49,72.345,2015
1,2015-02-01,102.4,82.5,88.1,85.4,106.28,72.509,2015
2,2015-03-01,100.7,82.9,88.1,85.7,112.70,72.809,2015
3,2015-04-01,103.0,83.3,88.3,85.9,115.28,73.169,2015
4,2015-05-01,101.5,83.7,88.0,86.0,109.25,73.441,2015


# 4. 거시경제변수 데이터

In [200]:
econ = pd.DataFrame(index=np.arange(2015, 2023))
econ

""
2015
2016
2017
2018
2019
2020
2021
2022


In [201]:
# KOSIS 데이터 합치기

econ['GDP성장률'] = gdp.values.flatten()
econ['원달러환율'] = usd.values.flatten()
econ['소비자물가지수증감'] = dcpi.values.flatten()

In [202]:
# ECOS 데이터 합치기

econ['생산자물가지수증감'] = dppi
econ = pd.concat([econ, ecos_y], axis=1)
econ['경제심리지수'] = ecos_m.groupby('연도').경제심리지수.mean()
#econ['선행종합지수'] = ecos_m.groupby('연도').선행종합지수.mean()
#econ['동행종합지수'] = ecos_m.groupby('연도').동행종합지수.mean()
#econ['후행종합지수'] = ecos_m.groupby('연도').후행종합지수.mean()
econ['뉴스심리지수'] = ecos_m.groupby('연도').뉴스심리지수.mean()
econ['주택매매가격지수'] = ecos_m.groupby('연도').주택매매가격지수.mean()

In [203]:
econ = econ.round(2)
print(econ.shape)
econ.to_csv('./data/거시경제데이터.csv')
econ

(8, 15)


,GDP성장률,원달러환율,소비자물가지수증감,생산자물가지수증감,무담보콜금리,CD91일,국고채3년,코스피종가,코스닥종가,경상수지,전산업생산지수,실업률,경제심리지수,뉴스심리지수,주택매매가격지수
2015,2.81,1131,0.7,-4.0,1.65,1.76,1.79,"1,961.31",682.35,"105,118.60",93.2,3.6,97.59,105.45,73.92
2016,2.95,1160.8,1.0,-1.8,1.34,1.49,1.44,"2,026.46",631.44,"97,923.70",96.0,3.7,95.08,96.24,75.88
2017,3.16,1131,1.9,3.5,1.26,1.44,1.80,"2,467.49",798.42,"75,230.90",98.5,3.7,99.78,106.92,76.86
2018,2.91,1100.2,1.5,1.9,1.52,1.68,2.10,"2,041.04",675.65,"77,466.50",100.1,3.8,97.83,99.70,78.57
2019,2.24,1165.4,0.4,0.0,1.59,1.69,1.53,"2,197.67",669.83,"59,676.10",101.1,3.8,94.09,96.00,79.69
2020,-0.71,1180.3,0.5,-0.5,0.70,0.92,0.99,"2,873.47",968.42,"75,902.20",100.0,4.0,79.18,96.71,82.71
2021,4.30,1144,2.5,6.4,0.61,0.85,1.39,"2,977.65","1,033.98","85,228.20",105.3,3.7,103.34,112.40,93.93
2022,2.61,1291.4,5.1,8.4,2.03,2.49,3.20,"2,236.40",679.29,"29,830.90",110.1,2.9,100.28,94.21,100.14
